Updating the SED files for MEATS to newer MAST standards

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import os
import glob
from astropy.table import Table
from astropy.io import ascii
from astropy.convolution import convolve, Box1DKernel
from scipy.interpolate import interp1d

import astropy.units as u
from astropy import time, coordinates as coord
import astropy.constants as const
from astropy.modeling import models, fitting
from astropy.coordinates import SkyCoord
from matplotlib.gridspec import GridSpec
from datetime import datetime
from astroquery.simbad import Simbad

#matplotlib set up
%matplotlib inline
from matplotlib import rcParams
rcParams["figure.figsize"] = (14, 5)
rcParams["font.size"] = 20

In [3]:
oldv = 1
version = 25


    STIS spectra

DOI = ‘10.17909/T9DG6F’

EQUINOX =’2000.2’ or RADESYS = 'ICRS'

HLSPID =’MUSCLES’

HLSPVER=’V1’

LICENSE =’CC BY 4.0’

LICENSURL= 'https://creativecommons.org/licenses/by/4.0/'

TIMESYS: Character string code (see FITS Standard) for the time scale of time-related keywords. Ex: 'UTC'

 

    Model spectra

The same set of metadata is required as STIS data except for EQUINOX/RADESYS and TIMESYS.

 

    SED data 

The same set of metadata is required as STIS data except for OBSERVAT.

MJD-BEG and MJD-END in the primary header. (* see below)

 

    XMM data

The same set of metadata is required as STIS data.

RA_TARG and DEC_TARG values are missing. Please include the correct RA/Dec values.

In [4]:
# stars = os.listdir('draft_hlsp/')
stars = ['toi-402'] #just newer stars
# print(stars)
stars = np.array(stars)
mask = stars != '.ipynb_checkpoints'
# print(mask)
stars = stars[mask]
print(stars)
print(len(stars))

['toi-402']
1


In [5]:
star_list = Simbad.query_objects(stars)
star_list

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,SCRIPT_NUMBER_ID
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,int32
HD 15337,02 27 28.3781,-27 38 06.736,14,14,0.009,0.012,90,A,O,2020yCat.1350....0G,1


In [6]:

newpath = 'fixed_hlsp/'
for star in stars:
    print(star)
    newdir = '{}{}'.format(newpath, star)
    if not os.path.exists(newdir):
       os.makedirs(newdir)
    
    oldv = 1
    if star == 'wasp-63':
        oldv = 2
    starpath = 'draft_hlsp/{}/'.format(star)
    specs = np.hstack((glob.glob('{}*stis*.fits'.format(starpath)),glob.glob('{}*cos*.fits'.format(starpath))))
    mods = glob.glob('{}*mod*.fits'.format(starpath))
    seds = glob.glob('{}*multi*.fits'.format(starpath))
    xrays = np.hstack((glob.glob('{}*cxo*.fits'.format(starpath)),glob.glob('{}*xmm*.fits'.format(starpath))))

    # print(specs)
    # print(mods)
    # print(seds)
    # print(xrays)
    
    
    starts = []
    ends = []

    for spec in specs:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        ra, dec = hdr['RA_TARG'], hdr['DEC_TARG']
        starts.append(hdr['EXPSTART'])
        ends.append(hdr['EXPEND'])
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('RADESYS' , 'ICRS'))
        hdr.append(('TIMESYS', 'UTC'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)
    
    for spec in mods:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)
    
    for spec in xrays:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        starts.append(hdr['EXPSTART'])
        ends.append(hdr['EXPEND'])
        hdr.append(('RA_TARG', ra))
        hdr.append(('DEC_TARG', dec))
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('RADESYS' , 'ICRS'))
        hdr.append(('TIMESYS', 'UTC'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)
    
    start = np.min(starts)
    end = np.max(ends)
    
    for spec in seds:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        hdr.append(('MJD-BEG', start))
        hdr.append(('MJD-END', end))
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('RADESYS' , 'ICRS'))
        hdr.append(('TIMESYS', 'UTC'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        for i in hdr:
            if hdr[i] in ['mod', 'Mod', 'MOD']:
                hdr[i] = 'MODEL'
        starname = hdr['TARGNAME']
        if starname[0:2].upper() == 'HD':
            starname = starname.replace('-', ' ')
        elif starname == 'kap1Cet':
            starname = '*kap01 Cet'
        simtab = Simbad.query_object(starname)
        coords = SkyCoord(simtab['RA'][0], simtab['DEC'][0], unit=(u.hourangle, u.deg))

        hdr['RA_TARG'] = coords.ra.deg
        hdr['DEC_TARG'] = coords.dec.deg
        
      
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)


toi-402


In [7]:
146-66

80

In [8]:
# Simbad.query_objects(stars)

In [9]:
len(stars)

1

In [10]:
for i in hdr:
    if hdr[i] in ['mod', 'Mod', 'MOD']:
        print (i, hdr[i])
        hdr[i] = 'MODEL'
        print (i, hdr[i])

In [11]:
hdr

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
TELESCOP= 'MULTI   '                                                            
INSTRUME= 'MULTI   '                                                            
GRATING = 'MULTI   '                                                            
TELESC00= 'MODEL   '                                                            
INSTRU00= 'LYA     '                                                            
GRATIN00= 'YOUNG   '                                                            
TELESC01= 'HST     '                                                            
INSTRU01= 'STS     '                                                            
GRATIN01= 'G140L   '        

In [12]:
c = SkyCoord(star_list['RA'][-1], star_list['DEC'][-1], unit=(u.hourangle, u.deg))
c

<SkyCoord (ICRS): (ra, dec) in deg
    (36.86824208, -27.63520444)>

In [13]:
c.dec.deg


-27.635204444444444

In [14]:
for star in stars:
    print(star)
    starpath = 'draft_hlsp/{}/'.format(star)
    seds = glob.glob('{}*multi*.fits'.format(starpath))
    for sed in seds:
        hdul = fits.open(sed)
        hdr = hdul[0].header
        starname = hdr['TARGNAME']
        print(starname)
        if starname[0:2].upper() == 'HD':
            starname = starname.replace('-', ' ')
        elif starname == 'kap1Cet':
            starname = '*kap01 Cet'
        simtab = Simbad.query_object(starname)
        coords = SkyCoord(simtab['RA'][0], simtab['DEC'][0], unit=(u.hourangle, u.deg))
        print(coords.ra.deg, coords.dec.deg)
        print(hdr['RA_TARG'], hdr['DEC_TARG'])
        # print(startab)

toi-402
TOI-402
36.86824208333333 -27.635204444444444
36.86770877243 -27.63655984225
TOI-402
36.86824208333333 -27.635204444444444
36.86770877243 -27.63655984225
TOI-402
36.86824208333333 -27.635204444444444
36.86770877243 -27.63655984225
TOI-402
36.86824208333333 -27.635204444444444
36.86770877243 -27.63655984225


In [15]:
# simtab

In [16]:
# Simbad.query_object('TOI-134')